# Player Scouting & Similarity Analysis
Find similar players and generate scouting reports for the 2024/25 season.

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import plotly.express as px
import plotly.graph_objects as go

DATA_DIR = "./data"

print("🔍 Player Scouting Pipeline")
print("="*60)

## 1. Load Player Data

In [ ]:
print("\n📂 Loading player data...")
player_profiles = pd.read_csv(os.path.join(DATA_DIR, "player_profiles_2024.csv"))

print(f"✓ Player profiles loaded: {len(player_profiles)} qualified players")
print(f"  Teams: {player_profiles['team_name'].nunique()}")
print(f"  Positions: {player_profiles['position'].unique()}")

## 2. Create Player Feature Vectors
Select relevant metrics for similarity comparison.

In [ ]:
print("\n" + "="*60)
print("CREATING PLAYER FEATURE VECTORS")
print("="*60)

# Define feature sets by position type
all_features_per90 = [
    'goals_per90', 'assists_per90', 'xG_per90', 'xA_per90',
    'shots_per90', 'key_passes_per90', 'xGChain_per90', 'xGBuildup_per90'
]

# Position-based feature weighting
position_groups = {
    'Forward': ['goals_per90', 'xG_per90', 'shots_per90', 'xGChain_per90'],
    'Midfielder': ['assists_per90', 'xA_per90', 'key_passes_per90', 'xGChain_per90', 'xGBuildup_per90'],
    'Defender': ['xGBuildup_per90', 'xGChain_per90']
}

# Normalize position labels
def normalize_position(pos):
    """Simplify position to Forward, Midfielder, or Defender."""
    pos = str(pos).upper()
    if 'FW' in pos or 'ST' in pos or 'CF' in pos:
        return 'Forward'
    elif 'MF' in pos or 'AM' in pos or 'DM' in pos or 'M' in pos:
        return 'Midfielder'
    elif 'DF' in pos or 'CB' in pos or 'FB' in pos or 'WB' in pos or 'D' in pos:
        return 'Defender'
    else:
        return 'Midfielder'

player_profiles['position_group'] = player_profiles['position'].apply(normalize_position)

print(f"✓ Position groups assigned:")
print(player_profiles['position_group'].value_counts())

## 3. Calculate Player Similarity
Use cosine similarity to find similar players within each position group.

In [ ]:
print("\n" + "="*60)
print("CALCULATING PLAYER SIMILARITY")
print("="*60)

def find_similar_players(player_name, n=5, position_group=None):
    """Find most similar players to a given player."""
    
    # Get player data
    player_data = player_profiles[player_profiles['player_name'] == player_name]
    
    if len(player_data) == 0:
        print(f"⚠️ Player '{player_name}' not found")
        return None
    
    player_data = player_data.iloc[0]
    player_pos = player_data['position_group']
    
    # Filter by position group
    if position_group:
        candidates = player_profiles[player_profiles['position_group'] == position_group].copy()
    else:
        candidates = player_profiles[player_profiles['position_group'] == player_pos].copy()
    
    # Remove the player themselves
    candidates = candidates[candidates['player_name'] != player_name]
    
    if len(candidates) == 0:
        print(f"⚠️ No candidates found for comparison")
        return None
    
    # Use position-relevant features
    feature_cols = all_features_per90
    
    # Prepare data
    X = candidates[feature_cols].fillna(0)
    player_vector = player_data[feature_cols].fillna(0).values.reshape(1, -1)
    
    # Standardize
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    player_scaled = scaler.transform(player_vector)
    
    # Calculate similarity
    similarities = cosine_similarity(player_scaled, X_scaled)[0]
    candidates['similarity'] = similarities
    
    # Get top matches
    top_matches = candidates.nlargest(n, 'similarity')
    
    return top_matches[['player_name', 'team_name', 'position', 'minutes', 
                        'goals_per90', 'assists_per90', 'xG_per90', 'similarity']]

# Example: Find similar players to some top scorers
print("\n🔍 Finding similar players...")

# Get top scorer
top_scorer = player_profiles.nlargest(1, 'goals_per90').iloc[0]['player_name']
print(f"\n📊 Players similar to {top_scorer}:")
similar = find_similar_players(top_scorer, n=5)
if similar is not None:
    print(similar.to_string(index=False))

## 4. Player Percentile Rankings
Calculate percentile ranks for each metric within position groups.

In [ ]:
print("\n" + "="*60)
print("CALCULATING PERCENTILE RANKINGS")
print("="*60)

# Calculate percentiles by position group
print("\n  Computing percentile ranks...")

percentile_cols = []
for col in all_features_per90:
    percentile_col = f'{col}_pct'
    player_profiles[percentile_col] = player_profiles.groupby('position_group')[col].rank(pct=True) * 100
    percentile_cols.append(percentile_col)

print(f"✓ Percentile rankings calculated for {len(all_features_per90)} metrics")

# Add overall rating (average of key percentiles)
key_metrics = {
    'Forward': ['goals_per90_pct', 'xG_per90_pct', 'shots_per90_pct'],
    'Midfielder': ['assists_per90_pct', 'xA_per90_pct', 'key_passes_per90_pct', 'xGChain_per90_pct'],
    'Defender': ['xGBuildup_per90_pct', 'xGChain_per90_pct']
}

def calculate_overall_rating(row):
    """Calculate position-specific overall rating."""
    pos_group = row['position_group']
    metrics = key_metrics.get(pos_group, all_features_per90)
    
    pct_metrics = [m + '_pct' if not m.endswith('_pct') else m for m in metrics]
    values = [row[m] for m in pct_metrics if m in row.index]
    
    return np.mean(values) if values else 0

player_profiles['overall_rating'] = player_profiles.apply(calculate_overall_rating, axis=1)

print(f"✓ Overall ratings calculated")

## 5. Generate Scouting Reports
Create detailed reports for top players.

In [ ]:
print("\n" + "="*60)
print("GENERATING SCOUTING REPORTS")
print("="*60)

def create_scouting_report(player_name):
    """Generate a detailed scouting report for a player."""
    
    player_data = player_profiles[player_profiles['player_name'] == player_name]
    
    if len(player_data) == 0:
        print(f"⚠️ Player '{player_name}' not found")
        return
    
    player = player_data.iloc[0]
    
    print(f"\n{'='*60}")
    print(f"SCOUTING REPORT: {player['player_name']}")
    print(f"{'='*60}")
    print(f"Team: {player['team_name']}")
    print(f"Position: {player['position']} ({player['position_group']})")
    print(f"Minutes Played: {player['minutes']:.0f} ({player['appearances']:.0f} appearances)")
    print(f"\n📊 OVERALL RATING: {player['overall_rating']:.1f}/100")
    
    print(f"\n⚽ ATTACKING METRICS (Percentile Rank vs {player['position_group']}s):")
    print(f"  Goals per 90: {player['goals_per90']:.2f} ({player['goals_per90_pct']:.0f}th percentile)")
    print(f"  xG per 90: {player['xG_per90']:.2f} ({player['xG_per90_pct']:.0f}th percentile)")
    print(f"  Assists per 90: {player['assists_per90']:.2f} ({player['assists_per90_pct']:.0f}th percentile)")
    print(f"  xA per 90: {player['xA_per90']:.2f} ({player['xA_per90_pct']:.0f}th percentile)")
    
    print(f"\n🎯 INVOLVEMENT METRICS:")
    print(f"  Shots per 90: {player['shots_per90']:.2f} ({player['shots_per90_pct']:.0f}th percentile)")
    print(f"  Key Passes per 90: {player['key_passes_per90']:.2f} ({player['key_passes_per90_pct']:.0f}th percentile)")
    print(f"  xGChain per 90: {player['xGChain_per90']:.2f} ({player['xGChain_per90_pct']:.0f}th percentile)")
    print(f"  xGBuildup per 90: {player['xGBuildup_per90']:.2f} ({player['xGBuildup_per90_pct']:.0f}th percentile)")
    
    # Find similar players
    print(f"\n🔍 SIMILAR PLAYERS:")
    similar = find_similar_players(player['player_name'], n=3)
    if similar is not None:
        for idx, sim_player in similar.iterrows():
            print(f"  • {sim_player['player_name']} ({sim_player['team_name']}) - {sim_player['similarity']:.2%} similar")
    
    print(f"\n{'='*60}\n")

# Generate reports for top players in each position
print("\n📋 Generating reports for top players by position...")

for pos_group in ['Forward', 'Midfielder', 'Defender']:
    top_player = player_profiles[player_profiles['position_group'] == pos_group].nlargest(1, 'overall_rating')
    if len(top_player) > 0:
        create_scouting_report(top_player.iloc[0]['player_name'])

## 6. Visualize Player Comparisons
Create scatter plots and radar charts for player analysis.

In [ ]:
print("\n" + "="*60)
print("CREATING VISUALIZATIONS")
print("="*60)

# Scatter plot: Goals vs Assists per 90
print("\n  Creating scatter plot...")

fig = px.scatter(
    player_profiles,
    x='goals_per90',
    y='assists_per90',
    color='position_group',
    size='minutes',
    hover_name='player_name',
    hover_data={
        'team_name': True,
        'position': True,
        'goals_per90': ':.2f',
        'assists_per90': ':.2f',
        'xG_per90': ':.2f',
        'minutes': True,
        'overall_rating': ':.1f'
    },
    title='Premier League Players: Goals vs Assists per 90 (2024/25)',
    labels={
        'goals_per90': 'Goals per 90',
        'assists_per90': 'Assists per 90',
        'position_group': 'Position'
    },
    width=1000,
    height=600
)

fig.update_traces(marker=dict(line=dict(width=0.5, color='white')))
fig.show()

print("✓ Scatter plot displayed")

# Radar chart for player comparison
def create_player_radar(player_name, comparison_players=None):
    """Create radar chart for player comparison."""
    
    player_data = player_profiles[player_profiles['player_name'] == player_name].iloc[0]
    
    # Use percentile ranks
    radar_metrics = [col.replace('_pct', '') for col in percentile_cols]
    percentiles = [player_data[f'{m}_pct'] for m in radar_metrics]
    
    # Simplify metric names for display
    display_names = [m.replace('_per90', '').replace('_', ' ').title() for m in radar_metrics]
    
    fig = go.Figure()
    
    # Add main player
    fig.add_trace(go.Scatterpolar(
        r=percentiles,
        theta=display_names,
        fill='toself',
        name=player_name,
        line=dict(color='#00ff87', width=2)
    ))
    
    # Add comparison players
    if comparison_players:
        for comp_name in comparison_players:
            comp_data = player_profiles[player_profiles['player_name'] == comp_name]
            if len(comp_data) > 0:
                comp_data = comp_data.iloc[0]
                comp_percentiles = [comp_data[f'{m}_pct'] for m in radar_metrics]
                
                fig.add_trace(go.Scatterpolar(
                    r=comp_percentiles,
                    theta=display_names,
                    fill='toself',
                    name=comp_name,
                    opacity=0.6
                ))
    
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, 100])),
        showlegend=True,
        title=f'Player Profile: {player_name} (Percentile Ranks)',
        width=800,
        height=600
    )
    
    return fig

# Example radar chart
print("\n  Creating radar chart...")
top_forwards = player_profiles[player_profiles['position_group'] == 'Forward'].nlargest(2, 'overall_rating')
if len(top_forwards) >= 2:
    player1 = top_forwards.iloc[0]['player_name']
    player2 = top_forwards.iloc[1]['player_name']
    
    fig = create_player_radar(player1, comparison_players=[player2])
    fig.show()
    print(f"✓ Radar chart comparing {player1} vs {player2}")

## 7. Save Results

In [ ]:
print("\n" + "="*60)
print("SAVING RESULTS")
print("="*60)

# Save enhanced player profiles with percentiles
player_profiles.to_csv(os.path.join(DATA_DIR, "player_profiles_enhanced_2024.csv"), index=False)
print(f"💾 Saved: player_profiles_enhanced_2024.csv")

# Create leaderboards
print("\n📊 Creating position-specific leaderboards...")

for pos_group in ['Forward', 'Midfielder', 'Defender']:
    leaderboard = player_profiles[player_profiles['position_group'] == pos_group].nlargest(20, 'overall_rating')
    filename = f"leaderboard_{pos_group.lower()}_2024.csv"
    leaderboard.to_csv(os.path.join(DATA_DIR, filename), index=False)
    print(f"💾 Saved: {filename}")

print("\n" + "="*60)
print("✅ PLAYER SCOUTING ANALYSIS COMPLETE!")
print("="*60)

print("\n📊 Summary:")
print(f"  Total players analyzed: {len(player_profiles)}")
print(f"  Forwards: {len(player_profiles[player_profiles['position_group'] == 'Forward'])}")
print(f"  Midfielders: {len(player_profiles[player_profiles['position_group'] == 'Midfielder'])}")
print(f"  Defenders: {len(player_profiles[player_profiles['position_group'] == 'Defender'])}")
print(f"\n  Top-rated Forward: {player_profiles[player_profiles['position_group'] == 'Forward'].nlargest(1, 'overall_rating').iloc[0]['player_name']}")
print(f"  Top-rated Midfielder: {player_profiles[player_profiles['position_group'] == 'Midfielder'].nlargest(1, 'overall_rating').iloc[0]['player_name']}")
print(f"  Top-rated Defender: {player_profiles[player_profiles['position_group'] == 'Defender'].nlargest(1, 'overall_rating').iloc[0]['player_name']}")